In [117]:
import osmnx as ox
import networkx as nx
import geopandas as gpd

import pandas as pd

import contextily
from IPython.display import GeoJSON

In [89]:
def geodata_to_df(country, city):

    G = ox.graph_from_place(city, network_type='bike')  # download raw geospatial data from OSM

    nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)
    nodes["city"], edges["city"] = city, city
    nodes["country"], edges["country"] = country, country

    # edges["lat_long"] = edges["geometry"].apply(lambda x: re.sub(r'[^0-9., ]', "", str([re.sub(r'[^0-9. ]', '', str(i)) for i in list(zip(x.xy[1], x.xy[0]))])))
    # edges["geometry"] = edges["geometry"].apply(lambda x: wkt.dumps(x))

    edges["highway"] = edges["highway"].apply(lambda x: ", ".join(x) if x.__class__.__name__=="list" else x)
    edges["name"] = edges["name"].apply(lambda x: ", ".join(x) if x.__class__.__name__=="list" else x)
    edges["maxspeed"] = edges["maxspeed"].apply(lambda x: ", ".join(x) if x.__class__.__name__ == "list" else x)
    edges["ref"] = edges["ref"].apply(lambda x: ", ".join(x) if x.__class__.__name__ == "list" else x)

    edges.fillna(-99, inplace=True)
    nodes.fillna(-99, inplace=True)
    edges["name"] = edges["name"].astype(str).replace("-99", None)
    
    nodes_and_edges = gpd.sjoin(edges, nodes, how="left", predicate="intersects")

    return G, nodes, edges, nodes_and_edges

In [90]:
G, nodes, edges, nodes_and_edges = geodata_to_df('Germany', 'Stuttgart')

In [92]:
# Get type of roads
road_types = nodes_and_edges['highway_left']

In [135]:
# Create a GeoDataFrame for intersections
intersections = gpd.GeoDataFrame(geometry=nodes.geometry)

# Get nodes with the highway=traffic_signals tag (intersections with lights)
highway_nodes = ox.features_from_place("Stuttgart", tags={"highway": "traffic_signals"})

# Get spots with bicycle parking
bicycle_parking = ox.features_from_place("Stuttgart", tags={"amenity": "bicycle_parking"})



In [143]:
# Get tram stops
transit_tram = ox.features_from_place("Stuttgart", tags={"railway": 'tram_stop'})

In [145]:
transit_tram.explore()

In [25]:
x = pd.DataFrame(road_types.value_counts())
x

,count
highway_left,
residential,43065
service,39267
path,37074
track,23836
secondary,9610
tertiary,8504
unclassified,4762
pedestrian,2688
living_street,2130
